In [7]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive

# 1. Conectar a Google Drive (Te pedirá permiso al ejecutar)
drive.mount('/content/drive')

# --- CONFIGURACIÓN ---
# IMPORTANTE: Asegúrate de que la carpeta en tu Drive se llame EXACTAMENTE así:
# Si tu carpeta se llama diferente, cambia la línea de abajo.
CARPETA_DATOS = "/content/drive/MyDrive/RodrigoReconomiento"

# Verificamos que la ruta exista para evitar errores
if not os.path.exists(CARPETA_DATOS):
    print(f"¡ERROR! No encuentro la carpeta en: {CARPETA_DATOS}")
    print("Revisa que hayas creado la carpeta 'ReconocimientoRodrigo' y dentro 'dataset' en tu Drive.")
else:
    print(f"✅ Carpeta encontrada: {CARPETA_DATOS}")

# 2. Generador de Imágenes
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Datos de Entrenamiento (Desde Drive)
print("\nCargando datos de ENTRENAMIENTO:")
train_generator = datagen.flow_from_directory(
    CARPETA_DATOS,
    target_size=(100, 100),
    batch_size=32,
    class_mode='binary',
    subset='training',
    classes=['otros', 'rodrigo'] # Asegura el orden: 0=otros, 1=rodrigo
)

# Datos de Validación (Desde Drive)
print("\nCargando datos de VALIDACIÓN:")
validation_generator = datagen.flow_from_directory(
    CARPETA_DATOS,
    target_size=(100, 100),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    classes=['otros', 'rodrigo']
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Carpeta encontrada: /content/drive/MyDrive/RodrigoReconomiento

Cargando datos de ENTRENAMIENTO:
Found 320 images belonging to 2 classes.

Cargando datos de VALIDACIÓN:
Found 79 images belonging to 2 classes.


In [8]:
# Definir el modelo (Red Neuronal Convolucional - CNN)
modeloCNN = tf.keras.models.Sequential([
    # Primera capa de convolución (detecta bordes y formas simples)
    # input_shape=(100, 100, 3) -> Imágenes de 100x100 a color (3 canales RGB)
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(100, 100, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    # Segunda capa
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    # Tercera capa
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    # Aplanar y conectar a la parte de clasificación
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation='relu'),

    # Capa de salida: 1 sola neurona (0 = Otros, 1 = Roberto)
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compilar el modelo
modeloCNN.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

print("✅ Modelo creado y compilado correctamente.")


from tensorflow.keras import layers, models
def build_model():
    model = models.Sequential([
        layers.Conv2D(32,(3,3),activation='relu',input_shape=(100,100,3)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(64,(3,3),activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(128,(3,3),activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

modelo = build_model()




✅ Modelo creado y compilado correctamente.


In [9]:
import tensorflow as tf
from PIL import ImageFile

# --- 1. ARREGLO PARA IMÁGENES ROTAS ---
# Esto evita que se detenga si una foto está dañada
ImageFile.LOAD_TRUNCATED_IMAGES = True

print("🚀 Iniciando entrenamiento (con protección contra imágenes rotas)...")

# --- 2. ENTRENAMIENTO ---
# Entrenamos por 20 épocas
try:
    historial = modeloCNN.fit(
        train_generator,
        epochs=20,
        validation_data=validation_generator
    )
    print("✅ ¡Entrenamiento finalizado con éxito!")

    # --- 3. EXPORTACIÓN ---
    print("\n📦 Exportando modelo...")
    modeloCNN.save('modelo_roberto.h5')

    # Crear carpeta y convertir
    import os
    if not os.path.exists('carpeta_salida'):
        os.makedirs('carpeta_salida')

    # Comando de sistema para convertir a TFJS
    print("Convertiendo a formato Web...")
    !pip install tensorflowjs
    !tensorflowjs_converter --input_format keras modelo_roberto.h5 carpeta_salida

    print("\n🎉 ¡TODO LISTO!")
    print("Ve al panel de archivos de la izquierda, busca 'carpeta_salida'")
    print("y descarga 'model.json' y el archivo '.bin'.")

except Exception as e:
    print("\n❌ Ocurrió un error:", e)
    print("Si el error persiste, es posible que el archivo ZIP se haya subido corrupto.")

🚀 Iniciando entrenamiento (con protección contra imágenes rotas)...
Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 57s 5s/step - accuracy: 0.5459 - loss: 0.6301 - val_accuracy: 0.8481 - val_loss: 0.2486
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.8947 - loss: 0.1965 - val_accuracy: 0.9620 - val_loss: 0.0621
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.9472 - loss: 0.0928 - val_accuracy: 0.9873 - val_loss: 0.0608
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.9907 - loss: 0.0329 - val_accuracy: 0.9873 - val_loss: 0.0338
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 1.0000 - loss: 0.0125 - val_accuracy: 1.0000 - val_loss: 0.0180
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.9973 - loss: 0.0078 - val_accuracy: 0.9747 - val_loss: 0.0930
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 1.0000 - loss: 0.0059 - val_accuracy: 1.0000 - val_loss: 0.0015
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step 

✅ ¡Entrenamiento finalizado con éxito!

📦 Exportando modelo...
Convertiendo a formato Web...
2025-12-03 22:06:10.643864: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764799570.684025   13992 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764799570.697413   13992 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764799570.732402   13992 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764799570.732835   13992 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than 

In [3]:
from google.colab import files
files.download('carpeta_salida/model.json')
files.download('carpeta_salida/group1-shard1of2.bin')
files.download('carpeta_salida/group1-shard2of2.bin')

FileNotFoundError: Cannot find file: modelo_web_rostro/model.json